In [2]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../..')
from modules.many_features import utils, constants, env
from modules.many_features.dt_agent import DTAgent
from sklearn.model_selection import train_test_split

In [3]:
SEED = 42
np.random.seed(SEED)

#### The data

In [4]:
# df = pd.read_csv('../../data/more_features/more_feats_correlated_noisy_2.csv')
# df = df.fillna(-1)
# df.head()
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
test_df = test_df.fillna(-1)
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.783006,112.011007,-1.000000,3.308648,348.014906,86.577465,-1.000000,4.082935,0,1.617594,131.868365,-1.000000,-1.000000,-1.000000,-1.000000,35.349018,-1.000000,7
1,12.609666,104.475099,5.445972,-1.000000,-1.000000,75.082198,-1.000000,5.038345,1,0.334722,97.568987,83.161880,-1.000000,7.147270,-1.000000,37.828999,-1.000000,3
2,7.940172,4.925400,3.255130,5.766738,209.031102,75.254082,-1.000000,3.165345,1,1.649478,2.082970,-1.000000,-1.000000,-1.000000,-1.000000,23.820517,-1.000000,4
3,7.785840,103.757096,2.874384,3.839597,-1.000000,81.873581,39.638010,2.852876,1,0.637843,127.412065,38.534760,51.437387,28.150784,73.296289,23.357519,-1.000000,5
4,12.359715,51.274141,0.178269,1.233844,474.589534,76.469181,49.839768,4.848900,1,0.621034,137.433107,55.314614,47.302738,3.808612,60.076211,37.079144,10.501658,4


In [5]:
X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_test, y_test = np.array(X_test), np.array(y_test)

#### The DT agent

In [6]:
# %%time
dt_agent = DTAgent(X_test, y_test)
test_df = dt_agent.test()
test_df.head()

Testing done.....


,episode_length,index,is_success,reward,terminated,trajectory,y_actual,y_pred
0,4.0,0.0,1.0,1.0,0.0,"[hemoglobin, mcv, ret_count, Inconclusive diag...",7.0,7.0
1,5.0,1.0,1.0,1.0,0.0,"[hemoglobin, gender, mcv, ferritin, Anemia of ...",3.0,3.0
2,4.0,2.0,1.0,1.0,0.0,"[hemoglobin, mcv, ferritin, Iron deficiency an...",4.0,4.0
3,4.0,3.0,1.0,1.0,0.0,"[hemoglobin, mcv, ret_count, Hemolytic anemia]",5.0,5.0
4,6.0,4.0,1.0,1.0,0.0,"[hemoglobin, gender, mcv, ferritin, tibc, Iron...",4.0,4.0


In [7]:
len(test_df), len(X_test)

(14000, 14000)

In [8]:
# test_df.to_csv('../../test_dfs/many_features/0.1/correlated/dt_test_df_corr.csv', index=False)

In [9]:
# test_df = pd.read_csv('../../test_dfs/many_features/dt_test_df_0.1.csv')
# test_df.head()

In [10]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

100.0

In [11]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

(4.002357142857143, 1.0)

In [12]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(1.0, 1.0, 1.0)

In [ ]:
# utils.plot_classification_report(test_df['y_actual'], test_df['y_pred'])

In [ ]:
# utils.plot_confusion_matrix(test_df['y_actual'], test_df['y_pred'])